In [126]:
import pickle
import sys
import math
from random import random, randint

In [125]:
with open('../data/test.pickle', 'rb') as fd:
    test_cases = pickle.load(fd)

with open('../data/map.pickle', 'rb') as fm:
    mp = pickle.load(fm)

with open('../data/dict.pickle', 'rb') as fd:
    dc = pickle.load(fd)

print('Initialized')

Initialized


In [174]:
def strictCmp(a, b, p):
    return 1. if a == b else 0

def rougeCmp(a, b, p):
    if len(a) != len(b) or len(a) < 1:
        return 0.
    a = '_' + a
    b = '_' + b
    n = len(a)
    f = []
    for i in range(n):
        f.append([ 0 ] * n)
    for i in range(1, n):
        for j in range(1, n):
            f[i][j] = max(f[i][j], f[i - 1][j], f[i][j - 1])
            if a[i] == b[j]:
                f[i][j] = max(f[i][j], f[i - 1][j - 1] + 1)
    n -= 1
    lcs = f[n][n]
    return 1. * lcs / max(n, 1.)

def genResStr(tot, rat1, rat2):
    return 'Precision %.6f ROUGE-L %.6f' % ( 1. * rat1 / tot, 1. * rat2 / tot )

def test(beg = 0, end = -1):
    if end == -1:
        end = len(test_cases)
    tot = 0
    res_strict = 0
    res_lcs = 0
    for i in range(beg, end):
        c = test_cases[i]
        ans = pinyin2text(c['pinyin'])
        tot += 1
        res_strict += strictCmp(ans, c['text'], c['pinyin'])
        res_lcs += rougeCmp(ans, c['text'], c['pinyin'])
        if i % 64 == 0:
            print('[Case 0 - %d] %s' % (i, genResStr(tot, res_strict, res_lcs)))

    print('[Fin] %s' % genResStr(tot, res_strict, res_lcs))

In [195]:
max_iter = 100
stable_iter = 3
eps = .000001
mul_co = 10.
innocent = math.exp(eps)

def getLnkVal(a, b, pin, direct):
    totl = 0
    for i in dc[pin]:
        if a in mp[direct] and i in mp[direct][a]:
            totl += mp[direct][a][i]
    if a in mp[direct] and b in mp[direct][a]:
        curl = mp[direct][a][b]
    else:
        curl = eps
    lnk_v = (curl + eps) / (totl + eps)
    # if totl > 0:
        # lnk_v *= math.log(totl)
    lnk_v *= math.log(23 + totl, 23)
    return lnk_v

def evaluateAns(py, ans):
    n = len(ans)
    s = 1.
    for i in range(0, n - 1):
        if i + 2 < n:
            s *= getLnkVal(ans[i], ans[i + 1], py[i + 1], 'fw')
            # print('%s -> %s fw %e' % (ans[i], ans[i + 1], getLnkVal(ans[i], ans[i + 1], py[i + 1], 'fw')))
        if i > 0:
            s *= getLnkVal(ans[i + 1], ans[i], py[i], 'bw')
            # print('%s <- %s fw %e' % (ans[i], ans[i + 1], getLnkVal(ans[i + 1], ans[i], py[i], 'bw')))
    return s

def randomFetchChar(prv_chr, py, direct, is_head, last_ans, p_cnt):
    wl = {}
    if not py in dc:
        return '<unk>'
    sum_p = 0
    max_p = 0
    totl = 0
    for i in dc[py]:
        if prv_chr in mp[direct] and i in mp[direct][prv_chr]:
            totl += mp[direct][prv_chr][i]
    mul_x = 16. / (totl + eps)
    for i in dc[py]:
        if prv_chr in mp[direct] and i in mp[direct][prv_chr]:
            # print('%s = %.5f / %d * 10 (%.5lf)' % (i, mp[direct][prv_chr][i], totl, mul_x))
            wl[i] = math.exp(mp[direct][prv_chr][i] * mul_x)
        elif i != py:
            wl[i] = innocent
        else:
            continue
        max_p = max(wl[i], max_p)
        sum_p += wl[i]
    if last_ans != '<todo>':
        if not last_ans in wl:
            wl[last_ans] = innocent
        if is_head:
            p_prv = max_p * p_cnt
        else:
            p_prv = wl[last_ans] * p_cnt
        wl[last_ans] += p_prv
        sum_p += p_prv
    chosen_p = random() * sum_p
    for i in wl:
        chosen_p -= wl[i]
        if chosen_p <= 0.:
            return i
    return '<err>'

def cmbAns(ans, plc = ''):
    return plc.join(ans[1 : -1])

def printAnsWithVal(py, ans):
    print('%s %e' % (cmbAns(ans), evaluateAns(py, ans)))

def pinyin2text(raw_line):
    a = [ '' ] + raw_line.strip().rstrip().split(' ') # For same id
    n = len(a) - 1
    ans = [ 'start' ] + [ '<todo>' ] * n + [ 'end' ]
    lrn_rat = 1
    for i in range(1, n + 1):
        ans[i] = randomFetchChar(ans[i - 1], a[i], 'fw', i == 1, ans[i], 0)
    best_ans = ans[:]
    best_ans_val = evaluateAns(a, ans)
    cur_val = best_ans_val
    max_rat = math.exp(30.)
    max_iter = 1000
    while lrn_rat < max_rat:
        cnt_stable = 0
        for cnt_iter in range(max_iter):
            b = ans[:]
            p = randint(1, n)
            d = [ 'fw', 'bw' ][randint(0, 1)]
            if d == 'fw':
                g_chr = randomFetchChar(ans[p - 1], a[p], 'fw', p == 1, ans[p], lrn_rat)
            else:
                g_chr = randomFetchChar(ans[p + 1], a[p], 'bw', p == n, ans[p], lrn_rat)
            b[p] = g_chr
            vb = evaluateAns(a, b)
            if vb > cur_val or random() < 1. / lrn_rat:
                cur_val = vb
                ans = b
                if cur_val > best_ans_val:
                    best_ans_val = cur_val
                    best_ans = ans[:]
                cnt_stable = 0
            else:
                cnt_stable += 1
                if cnt_stable > 5:
                    break
        if cnt_stable < 5:
            ans = best_ans[:]
            cur_val = best_ans_val
        lrn_rat *= 1.3
    printAnsWithVal(a, best_ans)
        #print('%s at %d' % (' '.join(ans[1 : n + 1]), cnt_iter))
    return ''.join(best_ans[1 : n + 1])



In [201]:

def pinyin2text(raw_line):
    a = [ '' ] + raw_line.strip().rstrip().split(' ') # For same id
    n = len(a) - 1
    ans = [ 'start' ] + [ '<todo>' ] * n + [ 'end' ]
    lrn_rat = 1
    for i in range(1, n + 1):
        ans[i] = randomFetchChar(ans[i - 1], a[i], 'fw', i == 1, ans[i], 0)
    best_ans = ans[:]
    best_ans_val = evaluateAns(a, ans)
    cur_val = best_ans_val
    max_rat = math.exp(10.)
    f = [ { 'start': { 'val': 1., 'text': 'start' } } ]
    for i in range(1, n + 1):
        f.append({})
        for v in dc[a[i]]:
            d = {
                    'val': 0.,
                    'text': v,
                    'from': '<unk>'
            }
            for jv in f[i - 1]:
                j = f[i - 1][jv]
                lnv = f[i - 1][jv]['val'] * getLnkVal(j['text'], v, a[i], 'fw')
                if i > 0:
                    lnv *= getLnkVal(v, j['text'], a[i - 1], 'bw')
                if i == n:
                    lnv *= getLnkVal('end', j['text'], a[i], 'bw')
                if lnv > d['val']:
                    d['val'] = lnv
                    d['from'] = j['text']
            f[i][v] = d
    ans = [ '' ] * (n + 1)
    ls_val = 0.
    ls_txt = ''
    for jv in f[n]:
        if f[n][jv]['val'] > ls_val:
            ls_val = f[n][jv]['val']
            ls_txt = jv
    ans[n] = jv
    for i in range(n - 1, 0, -1):
        ls_txt = f[i + 1][ls_txt]['from']
        ans[i] = ls_txt
    return ( ''.join(best_ans[1 : n + 1]), best_ans, best_ans_val)



In [202]:
print(pinyin2text('ben zhan li zu yu mei li jian he zhong guo dui quan qiu hua ren fu wu shou bei mei fa lv bao hu'))
evaluateAns([ '' ] + 'ben zhan li zu yu mei li jian he zhong guo dui quan qiu hua ren fu wu shou bei mei fa lv bao hu'.split(' '), 
            toArr('本站立足于美利坚合众国对全球华人服务受北美法律保护'))

('本站立足于美丽健和中国队全球化人负物受北美发旅保护', ['start', '本', '站', '立', '足', '于', '美', '丽', '健', '和', '中', '国', '队', '全', '球', '化', '人', '负', '物', '受', '北', '美', '发', '旅', '保', '护', 'end'], 3030.0098049439375)


38.25964985900712

In [191]:
print(evaluateAns([ '' ] + 'wo mei you shuo hua'.split(' '), 
            toArr('我没有说话')))
print(evaluateAns([ '' ] + 'wo mei you shuo hua'.split(' '), 
            toArr('我没有数化')))

3434.6806608301295
2387.1834274196317


In [175]:
test()

虽然已经解决了建立新积分方法的首要问题 1.093662e-10
[Case 0 - 0] Precision 1.000000 ROUGE-L 1.000000
建立了交易办机上的策都理论 3.151778e-07
后面我们将成具有着中性质的含数为可测韩束 3.971523e-11
下面引入可测韩束的改年 1.464052e-05
可测韩束的有限客家行 8.924227e-08
其核处处受连喊数列的空置受连定力 8.477688e-13
正名积分与其现校还顺序 5.958861e-10
在车一些控制售连定力的应用 1.199009e-07
都着自己也可以列车并加以证明 3.106975e-07
所以这两个寒暑期和处处相等 3.384697e-06
今天也是好天气 5.677539e-03
有在投篮嘛 6.101629e-03
你会改吧 1.474736e-02
王道整土 9.064241e-04
不存在的 3.260735e-01
为一直盯游项 6.275642e-06
相向记 2.616003e-03
直到你的希望习蔑为制 7.803005e-09
非常心险费场每位 1.349669e-05
以非重天啊我 4.147461e-03
今天也是好天气 5.677539e-03
有在投篮嘛 6.101629e-03
你会改吧 1.474736e-02
王道整土 9.064241e-04
不存在的 3.260735e-01
为一直盯游项 6.275642e-06
相向记 2.616003e-03
直到你的西忘熄灭未知 2.513135e-09
非常心险费场每位 1.349669e-05
以非重天啊我 4.147461e-03
这就是我们的契机 6.930661e-04
入国企及有颜色那一定是成色 9.000588e-10
董先生联人好不好啊 2.465812e-04
中央已经决定了 1.148725e-02
这个世界太乱 4.293494e-03
怪怪展好 6.094049e-02
你为什么这么熟练啊 1.114293e-03
明明是我现来的 8.302285e-04
感受和言之地的愤怒 9.066353e-07
中雨来了各有前途的家和 1.570161e-05
严格审核结款人还款能力 9.560127e-05
从严防控新大风险 7.796353e-03
离婚一年内贷款

有限装太自动机 8.597887e-06
黑云压成成渝粹 2.404821e-04
和方因小切需行 2.331343e-05
东风夜访华签署 6.546459e-05
草色要看今却无 5.323278e-05
很多人参与了此条编辑 1.496257e-04
人工智能是一门游区的科成 3.439712e-06
大作业要解不完了 6.421729e-04
计算机系没有吸管 2.702661e-07
嗖够把把对我们很好 3.421296e-03
落调试射中有明的小吃 7.252056e-06
起来极寒交迫的努力 2.490166e-06
起来全市接受苦的人 3.805871e-05
满腔的热血衣警匪疼 1.920122e-05
要为真理而斗争 1.419470e-04
就是家大哥罗华刘说 1.627487e-06
努力门起来起来 6.475003e-05
不要说我们一无所有 2.992130e-03
我们要做天下的主任 3.358527e-05
这是最后的都正 6.214763e-03
团结起来到明天 1.146704e-01
英特纳兄能而就一定要实现 5.975533e-06
上面是谁把过几个都铁出来了 2.011896e-04
那我要不要铁一手过个呢 2.918854e-06
算了孩失补贴了 6.570281e-04
绰晰可持 2.158730e-03
这位同学情逐一 1.127137e-03
上面的内首过几个 1.369844e-04
是没写出来的呢 4.106630e-04
所以不要说什么超细落 7.564725e-04
那我该说电沙 7.395539e-04
人文是冠军 1.817568e-04
工业工程是冠军 8.601200e-06
人文和工业工程必将会是决赛 2.617504e-05
规吸溜子般很强 1.039514e-04
所以我们一定能报记 2.591547e-03
作业余淑凤州 3.165599e-04
五是人非实施修 7.121121e-07
利时的车轮在年国 4.016400e-05
我们将网和出去 7.215929e-04
波浪时前进 2.776616e-04
落选时尚生 5.003935e-03
就令后不如中年为期 1.546110e-05
实上只有一中英雄主义 8.486458e-07
发现了生活的真相 1.204188e-03
依然热爱他

所以我们是老乡 6.502951e-03
阿 4.043302e-02
我要高思呢涯 7.524385e-03
清风不是自 4.509235e-03
和古銮犯数 2.201498e-02
是不杀一人潜力不流行 1.245012e-07
本站记者跑的会 9.026591e-03
泾嗽和大杀野县市人领题 2.464370e-08
系稳嗽和大厦将开放传说重物娃食单 7.709041e-09
搜和有细解拍客盟经领推出新种丑丑化 3.152188e-13
您的好有苗行人称公布获了利于网 1.607365e-10
您的重污缪缪向持调业生的比卡求 1.880101e-06
您这个人而打该系还找项 5.749328e-06
清华大学家防验与行动 9.694262e-05
赞我比出差目 6.186647e-04
大爱珞甜食 3.539617e-03
这是一个很长很长很长很长的养力输车 1.213544e-06
我在电量输资人生 6.021731e-04
而有些人已经在电然属子与周了 2.182857e-07
迟堡套不吐葡萄皮 1.508218e-04
不持普讨到吐葡萄皮 1.721183e-06
我好恶但是没有效撬篪 2.979039e-05
看了一些阳历数据我觉得我要挂科了 3.157313e-07
每个人都是理性的 1.224053e-03
[Case 0 - 704] Precision 0.144681 ROUGE-L 0.632470
能不能给我好一点的情的 1.809278e-03
上善若说 7.867935e-03
谢谢大家来参加我的生日 4.075184e-03
白洋作是一个可爱的性作 3.202866e-06
我的桌上有一幅减至 2.318103e-04
我的朋友送了我一盆绿植作为生日礼物 7.078700e-09
大家喜欢听的个多种多样 8.019275e-07
有没有一首个让你想起一个人 8.714098e-04
丝绸之路上的浪漫故事被市人船长 7.076466e-09
爱一个人这样容易 3.982424e-04
很一个人这样容易 2.957785e-04
进校被中营者明说 1.077699e-05
沸腾的条件事达到非典并持续吸热 5.400468e-11
我爱学习学习时我会了 5.185543e-04
人间四月房费金 1.420637e-04
电量数字人生 4

KeyError: 'qv'

In [135]:
def toArr(s):
    x = [ 'start' ]
    for i in range(len(s)):
        x.append(s[i])
    x.append('end')
    return x